0. 각 metric_prjct 별로 데이터 통합, score 계산 결과 통합
1. 각 metric_prjct 별로 mape, r2 score 값들의 분포 확인
    - 각각에 대한 histogram & 2차원 histogram
    - (각 metric_prjct 별로 튀는 부분에 대한 구체적 EDA) 
    - (각 metric_prjct 별로 scatter plot에서 데이터 hue e.g. 단위, 특징 등)
    - 각 metric_prjct의 2차원 scatter plot을 한 곳에 그리기
2. 각 item label 별로 prjct 별 score 비교
    - mape, r2 score 각각
    - (각각 별로 더 높은 prjct 표시)

In [2]:
import numpy as np 
import pandas as pd 
import matplotlib as mpl
import matplotlib.pyplot as plt 
import missingno as msno
import seaborn as sns 
import os

In [3]:
import functools, itertools
from sklearn.model_selection import train_test_split, cross_val_score, cross_val_predict
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_percentage_error as mape
from sklearn.metrics import mean_squared_error as mse

In [4]:
import matplotlib_inline.backend_inline

matplotlib_inline.backend_inline.set_matplotlib_formats("png2x")
# 테마 설정: "default", "classic", "dark_background", "fivethirtyeight", "seaborn"
mpl.style.use("fivethirtyeight")
# 이미지가 레이아웃 안으로 들어오도록 함
mpl.rcParams.update({"figure.constrained_layout.use": True})

In [5]:
import matplotlib.font_manager as fm
font_list = fm.findSystemFonts(fontpaths=None, fontext='ttf')
[fm.FontProperties(fname=font).get_name() for font in font_list if 'D2C' in font]
plt.rc('font', family='D2Coding')
mpl.rcParams['axes.unicode_minus'] = False

In [6]:
DATASET_DIR = '/home/doeun/code/AI/ESTSOFT2024/workspace/dataset/'
load_dir = 'america_big_cities_health_inventory'
file_name = 'BigCitiesHealth.csv'
RSLT_DIR = '/home/doeun/code/AI/ESTSOFT2024/workspace/1.project1_structured/BCHI/processed/'
pvtb_dir = RSLT_DIR + 'pvtb/'

## FUNCTIONS

- preprocess

In [7]:
def count_key_opt(data:pd.DataFrame,key,opt):
    rslt = dict()
    labels = data[key].unique()
    form = data[opt].value_counts().sort_values(ascending=False)
    form.iloc[:] = 0

    for feat in labels:
        cond = data[key]==feat
        val = form.copy()
        temp = data.loc[cond,opt].value_counts()
        val.loc[temp.index] = temp
        rslt[feat] = val

    return pd.DataFrame(rslt).T

In [8]:
import functools

def cond_check_dict(data=pd.DataFrame,val_dict=dict):
    cond_list=[
        data[col] == val
        for col, val in val_dict.items()
    ]
    return functools.reduce(lambda x,y: x & y, cond_list)

- plot

In [9]:
import re

def choose_split_point(word_len,space,ths):
    # 윗 줄에 space 만큼 공백이 있고, 한 줄의 길이가 ths로 제한 되어있을 때
    # 어떤 지점에서 단어를 끊어줄지 정하기
    # |-------ths-------|
    # |-space-|---------|-space-|------| : word
    #         |-------ths-------|
    print(word_len,space,ths)
    if word_len < ths + space :
        if abs(word_len/2 -ths) <= abs(word_len/2-space) :
            return word_len-ths
        else :
            return word_len - space if word_len < 2 * space else space
    else :
        return ths if word_len - (ths + space) < 0.3 * ths else space

def minimize_seq_idx_np(domain:np.array,func):
    vfunc = np.vectorize(func)
    temp = np.argsort(vfunc(domain))
    return temp[0]

def modify_strlen_ths(last,new,ths=16):
    front = len(last)
    space = ths - (1+front)
    if len(new) < space :
        rslt = [last + ' ' + new]
    else :
        if len(new) < ths:
            rslt = [last, new]
        else:
            cut = choose_split_point(len(new),space-1,ths-1)
            new_h, new_e = new[:cut]+'-', new[cut:]
            if cut < ths-1 :
                rslt = modify_strlen_ths(last+' '+new_h,new_e)
            else :
                rslt = [last] + modify_strlen_ths(new_h,new_e) 
    return rslt

def str_cutter(sentnc, ths = 16):
    words= sentnc.split(' ')
    rslt, pnt = [''], 0
    while pnt < len(words):
        last = '' if len(rslt)==0 else rslt[-1]
        next_ele = modify_strlen_ths(last,words[pnt],ths)
        rslt = rslt[:-1] + next_ele
        pnt += 1
    return '\n'.join(rslt)[1:]
#알고리즘 때문에 맨 앞에 빈칸 하나 들어가게 되는 이슈 있음

print(str_cutter('Racial Segregation Indices | Racial Segregation, White and Hispanic', 13))



Racial
Segregation
Indices |
Racial
Segregation,
White and
Hispanic


In [10]:
def choose_plot_grid(n:int,r_max=8,c_max=17,res_ths=2):
    #ver2
    r_min = np.ceil(n/c_max)
    sppt = np.arange(r_min,r_max+1) #need error process
    col_nums = np.ceil(n/sppt)
    res = col_nums * sppt -n
    min_idx = np.where((res==np.min(res)) | (res <= res_ths))[0]
    row_cand, col_cand = sppt[min_idx], col_nums[min_idx]
    if len(min_idx) > 1 :
        res = np.abs(row_cand-col_cand)
        i = np.where(res==np.min(res))[0][0]
    else : i = 0
    return int(row_cand[i]), int(col_cand[i])

def pair_plot_feat_hue(fig,axes,data:dict,pair_plot,axis_share=False,hue_label_dict=None, **kwargs):
    #ver2
    if (fig is None) or (axes is None) :
        num_r, num_c = choose_plot_grid(len(data))
        fig, axes = plt.subplots(num_r,num_c,figsize=(4*num_c,4*num_r),sharex=axis_share,sharey=axis_share)
    for n,key in enumerate(data.keys()):
        ax = axes.flatten()[n]
        plt.setp(ax.get_xticklabels(),ha = 'left',rotation = 90)
        if n >= len(data) : continue
        pair_plot(x=data[key][0], y = data[key][1],ax =ax, **kwargs)
        feat_name = str(key) 
        if hue_label_dict: color = 'b' if hue_label_dict[feat_name] else 'k'
        else : color = 'k'
        ax.set_xlabel(str_cutter(feat_name,20),loc='left',fontsize = 8.3,color=color)
    return fig,axes

In [11]:
## FUNCTIONS - DATA- SCORE REG RSLT

def make_reg_score_dict(y_actual,y_pred,base_val):
    rmse_model, rmse_base = np.sqrt(mse(y_actual,y_pred)), np.sqrt(mse([base_val]*len(y_actual),y_actual))
    mape_model, mape_base = mape(y_actual,y_pred), mape([base_val]*len(y_actual),y_actual)
    r2_model, r2_base = r2_score(y_actual,y_pred), 0
    
    return {
        'rmse' : [rmse_model, rmse_base],
        'mape' : [mape_model, mape_base],
        'r2_score' : [r2_model,r2_base]
    }

def print_reg_score_dict(name,dict_score):
    print('{}\nr2 score : {:.5f}'.format(name,dict_score['r2_score'][0]))
    print('rmse_model : {:.5f} / rmse_base : {:.5f}\t'.format(*dict_score['rmse']),
          'mape_model : {:.5f} / mape_base : {:.5f}\t'.format(*dict_score['mape']))

def make_reg_score_dict_cols(target_sample,dict_df,dict_train_test,dict_rslt,print_plot=False):
    dict_score = dict()
    for col in target_sample:
        train_y = dict_df[col]['train'][1]
        valid_y = dict_train_test[col][3]
        y_pred = dict_rslt[col]['valid']
        dict_score[col] = make_reg_score_dict(valid_y,y_pred,np.mean(train_y))
        if print_plot :
            print_reg_score_dict(col,dict_score[col])
            print('-'*150)
    return dict_score

In [12]:
## FUNCTIONS - DATA- PLOT REG RSLT
from itertools import repeat, chain

def scatter_reg_rslt(dict_train_test,dict_rslt,dict_score): #set_iput
    target_sample = list(dict_rslt.keys())
    data_plot ={
        col : (dict_train_test[col][3], dict_rslt[col]['valid'])
        for col in target_sample 
    }
    data_line = {
        col : (dict_train_test[col][3],dict_train_test[col][3])
        for col in target_sample 
    }
#    fig,axes = plt.subplots(3,3,figsize=(12,12))
#    fig,axes = pair_plot_feat_hue(fig=fig,axes=axes,data=data_line,
    fig,axes = pair_plot_feat_hue(fig=None,axes=None,data=data_line,
                                  pair_plot=sns.lineplot,lw=0.3)
    #fig.set_size_inches(12,8, forward=True)
    fig,axes = pair_plot_feat_hue(fig=fig,axes=axes,data=data_plot,
                                  pair_plot=sns.scatterplot,s=5,alpha=0.65)

    for n,key in enumerate(data_plot.keys()):
        ax = axes.flatten()[n]
        ax.set_ylabel('')
        ax.set_title(key,fontsize=12)
        ax.set_xlabel('rmse_model : {:.3f} | rmse_base : {:.3f}\nr2 score : {:.3f} {:>35}\n'.format(*dict_score[key]['rmse'],dict_score[key]['r2_score'][0],
            f'n = {len(dict_train_test[key][0])}'), fontsize=10, ha ='left')

    for ax in axes.flatten():
        plt.setp(ax.get_yticklabels(),rotation = 0, fontsize = 9)
        plt.setp(ax.get_xticklabels(),ha ='center',rotation = 0, fontsize = 9)
    
    return fig,axes

def plot_reg_score(dict_train_test,dict_rslt,dict_score,target_sample):
    data_plot ={
        col : (dict_train_test[col][3], dict_rslt[col]['valid'])
        for col in target_sample 
    }
    fig,axes = plt.subplots(len(data_plot),3,figsize=(15,20))
    for n, col in enumerate(data_plot.keys()):
        #quo, rem = divmod(n,3)
        ax1, ax2, ax3 = axes[n][0], axes[n][1], axes[n][2]
        test_y, y_pred = data_plot[col]
        train_y = dict_train_test[col][2]

        sns.histplot(test_y,label='actual',ax=ax1,alpha=0.5)
        sns.histplot(y_pred,label='knn_pred',ax=ax1,alpha=0.5)
        ax1.legend(fontsize=9)

        sns.histplot(test_y-np.mean(train_y),ax=ax2, label = 'baseline',alpha=0.5)
        sns.histplot(test_y-y_pred,ax=ax2, label = 'knn_pred',alpha=0.5)
        ax2.legend(fontsize=9)

        df_score = pd.DataFrame(dict_score[col]).T[[1,0]]
        xs = list(chain.from_iterable(repeat(val,2) for val in df_score.index))
        ax3r = ax3.twinx()
        sns.barplot(x=xs[:2],y=list(df_score.values.reshape(-1))[:2],
                    hue = ['knn_pred','base']*1,ax=ax3,alpha=0.65,legend=False)
        sns.barplot(x=xs[2:],y=list(df_score.values.reshape(-1))[2:],
                    hue = ['knn_pred','base']*2,ax=ax3r,alpha=0.8,legend=False)
        ax3.set_yscale('log')
        ax3r.set_ylim([0.0,1.15])
        ax3r.bar_label(ax3r.containers[0], fontsize=8, fmt='%.4f')
        ax3r.bar_label(ax3r.containers[1], fontsize=8, fmt='%.4f')
        ax3.grid(False)
        ax3r.grid(False)
        ax3r.set_yscale('linear')    
        ax3.bar_label(ax3.containers[0], fontsize=8, fmt='%.4f')
        ax3.bar_label(ax3.containers[1], fontsize=8, fmt='%.4f')
        #ax3.axvline()

        ax1.set_title(str_cutter(col,50),fontsize=15,loc='left',ha='left')
        ax2.xaxis.set_label_coords(-0.02, -0.15)
        ax2.set_xlabel('rmse_model : {:.3f} | rmse_base : {:.3f}\nr2 score : {:.3f} {:>48}\n'.format(*dict_score[col]['rmse'],dict_score[col]['r2_score'][0],
            f'n = {len(dict_train_test[col][0])}'), fontsize=10,ha ='left')
        #ax2.xaxis.set_label_position('left')
        ax1.set_xlabel('')
        ax3.set_xlabel('')
        ax1.set_ylabel('count',fontsize =10) 
        ax2.set_ylabel('count',fontsize =10) 
        plt.setp(ax3.get_yticklabels(),rotation = 0, fontsize = 9, color='#333333')
        plt.setp(ax3r.get_yticklabels(),rotation = 0, fontsize = 9)

    for ax in axes.flatten():
        plt.setp(ax.get_yticklabels(),rotation = 0, fontsize = 9)
        plt.setp(ax.get_xticklabels(),ha ='center',rotation = 0, fontsize = 9)

    return fig, axes

In [13]:
## FUNCTIONS - FILE I/O

import pickle

def save_pkl(save_dir,file_name,save_object):
    if not os.path.exists(save_dir): os.mkdir(save_dir)
    file_path = os.path.join(save_dir,file_name)
    with open(file_path,'wb') as f:
        pickle.dump(save_object,f)

## Main

In [15]:
os.listdir(RSLT_DIR)

['knn_canberra7_cand',
 'knn_canberra5_else',
 'knn_custom7_cand',
 'knn_braycurtis5_cand',
 'knn_cand',
 'pivot_racesex.csv',
 'knn_braycurtis7_cand',
 'geo_strat_info.csv',
 'knn_braycurtis7_else',
 'knn_custom5_cand',
 'knn_custom7_else',
 'knn_canberra5_cand',
 'knn_braycurtis5_else',
 'pvtb',
 'knn_custom5_else',
 'knn_cosine5_cand',
 'knn_canberra7_else',
 'group_statistics.csv',
 'knn_else']

In [ ]:
files = list(filter(lambda x : ('knn' in x) & ('5' in x) ,os.listdir(RSLT_DIR)))
file_dict = dict()
for filename in files:

In [ ]:
knn_path = os.path.join(RSLT_DIR,knn_dir)
work_idx = 0
n_work = 5 

for work_idx in tqdm(range(1,5)):

    work_name = '{}_{}'.format(work_idx, n_work)
    target_sample = target_cols[work_idx::n_work]
    work_name = '{}_{}'.format(work_idx,n_work) 
    print(f'work : {work_name}')

    pvtb_encoded['city_idx'] = pvtb_encoded['geo_label_city'].apply(lambda x : city_list.index(x))
    temp = list(pvtb_encoded.columns)
    pvtb_encoded_whole = pvtb_encoded[['city_idx']+temp[1:-1]]
    test_df = pvtb_encoded[info_cols+target_cols]


    files = list(filter(lambda x : work_name in x,os.listdir(knn_path)))
    file_dict = dict()
    for filename in files:
        if filename[-4:] != '.pkl' : continue
        if 'knn' in filename : continue
        print(filename)
        file_path = os.path.join(knn_path,filename)
        with open (file_path,'rb') as f:
            file_dict[filename[:-5-len(work_name)]] = pickle.load(f)

    dict_rslt = file_dict['dict_rslt']
    target_sample= list(dict_rslt.keys())

    rslt_form = test_df[info_cols+target_sample]

    for col in target_sample:
        cond = test_df[col].isna()
        loaded = dict_rslt[col]['target']
        assert np.sum(cond) == len(loaded)  
        rslt_form.loc[cond,col] = loaded

    file_name = 'pvtb_filled_knn_{}_{}.csv'.format(work_idx,10)
    save_dir = os.path.join(RSLT_DIR,knn_dir,'PROCESSED')
    if not os.path.exists(save_dir): os.mkdir(save_dir)
    rslt_form.to_csv(os.path.join(save_dir,file_name))

    ### check score for loaded data

    dict_rslt=file_dict['dict_rslt']
    dict_train_test=file_dict['dict_train_test']

    dict_df = dict()
    for col in target_sample:
        temp = test_df[info_cols+[col]]
        cond_na = temp.isna().any(axis=1)
        dict_df[col] = {
            'train' : [temp.loc[~cond_na,info_cols], temp.loc[~cond_na,col]],
            'target' : [temp.loc[cond_na,info_cols], cond_na],
        }

    ## check score
    dict_score = make_reg_score_dict_cols(target_sample,dict_df,dict_train_test,dict_rslt,print_plot=True)